# Dense Optical Flow in OpenCV

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow

This function computes a dense optical flow using the Gunnar Farneback's algorithm.

Here are the parameters for the function and what they represent:
   
* prev first 8-bit single-channel input image.
* next second input image of the same size and the same type as prev.
* flow computed flow image that has the same size as prev and type CV_32FC2.
* pyr_scale parameter, specifying the image scale (\<1) to build pyramids for each image
    * pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
    
* levels number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
* winsize averaging window size
    * larger values increase the algorithm robustness to image
* noise and give more chances for fast motion detection, but yield more blurred motion field.
* iterations number of iterations the algorithm does at each pyramid level.
* poly_n size of the pixel neighborhood used to find polynomial expansion in each pixel
    * larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
* poly_sigma standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

In [25]:
import cv2 as cv 
import numpy as np 

In [26]:
cap = cv.VideoCapture(0)
ret,frame1 = cap.read()
previous_image = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

In [27]:
hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255 

In [28]:
while True:
  ret,new_frame = cap.read()
  new_image = cv.cvtColor(new_frame,cv.COLOR_BGR2GRAY)

  flow_detect = cv.calcOpticalFlowFarneback(previous_image,new_image,None,0.5,3,15,3,5,1.2,0)
  
  magnitude,angle  = cv.cartToPolar(flow_detect[:,:,0],flow_detect[:,:,1],angleInDegrees=True)

  hsv_mask[:,:,0] = angle/2
  hsv_mask[:,:,2] = cv.normalize(magnitude,None,0,255,cv.NORM_MINMAX)

  bgr = cv.cvtColor(hsv_mask,cv.COLOR_HSV2BGR)
  cv.imshow('Frame',bgr)

  k = cv.waitKey(10) & 0xFF

  if k == 27:
    break 

  previous_image = new_image

cap.release()
cv.destroyAllWindows()
